In [58]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import os 
import matplotlib.pyplot as plt

In [59]:
os.chdir( "C:\\Users\\vyago\\Desktop\\Maestría Ciencias de Datos\\07-DMEYF" )

dataset = pd.read_csv("./datasets/competencia2_2022.csv.gz", low_memory=False)


enero = dataset.loc[dataset["foto_mes"]==202101]
marzo = dataset.loc[dataset["foto_mes"]==202103]


In [60]:
enero=enero.fillna(0) # RELLENO CON 0 LOS NULOS, LO NECESITA LA RED
marzo = marzo.fillna(0)

In [63]:
marzo["clase_binaria"] = marzo["clase_ternaria"].apply(lambda x: 1 if x=="CONTINUA" else 0) #CLASE COMBINADA
enero["clase_binaria"] = enero["clase_ternaria"].apply(lambda x: 1 if x=="CONTINUA" else 0)
drop = ["clase_binaria","clase_ternaria","foto_mes","numero_de_cliente"]
X_train = enero[enero.columns.drop( drop)]
y_train = enero["clase_binaria"]
X_test = marzo[marzo.columns.drop(drop)]
y_test = marzo["clase_binaria"]


In [64]:
# ESCALADO DE FEATURES
train_max = X_train.max()
train_min = X_train.min()


train_data = (X_train - train_min)/(train_max - train_min)
test_data = (X_test - train_min)/(train_max - train_min)


In [65]:
train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

In [66]:
train_labels = y_train.values.astype(bool)
test_labels = y_test.values.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

In [67]:
normal_train_data.shape

TensorShape([159828, 152])

## Modelo

In [68]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(64,activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(64,activation="relu"),
      layers.Dense(152, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [69]:
autoencoder.compile(optimizer='adam', loss='mae')

In [ ]:
history = autoencoder.fit(normal_train_data, normal_train_data, 
          epochs=300, 
          batch_size=512,
          validation_data=(test_data, test_data),
          shuffle=True)

Epoch 1/300
313/313 [==============================] - 1s 3ms/step - loss: 0.0131 - val_loss: 0.0131
Epoch 2/300
313/313 [==============================] - 1s 3ms/step - loss: 0.0129 - val_loss: 0.0129
Epoch 3/300
313/313 [==============================] - 1s 3ms/step - loss: 0.0128 - val_loss: 0.0128
Epoch 4/300
313/313 [==============================] - 1s 3ms/step - loss: 0.0126 - val_loss: 0.0125
Epoch 5/300
313/313 [==============================] - 1s 4ms/step - loss: 0.0123 - val_loss: 0.0123
Epoch 6/300
313/313 [==============================] - 1s 3ms/step - loss: 0.0122 - val_loss: 0.0122
Epoch 7/300
313/313 [==============================] - 1s 3ms/step - loss: 0.0121 - val_loss: 0.0122
Epoch 8/300
313/313 [==============================] - 1s 3ms/step - loss: 0.0121 - val_loss: 0.0121
Epoch 9/300
313/313 [==============================] - 1s 3ms/step - loss: 0.0121 - val_loss: 0.0121
Epoch 10/300
313/313 [==============================] - 1s 4ms/step - loss: 0.0120 - val_lo

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

## Perdida de reconstrucción

In [ ]:
reconstructions = autoencoder.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

plt.hist(train_loss[None,:], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
threshold = np.mean(train_loss) + np.std(train_loss)